In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16027361
paper_name = 'serviene_nagy_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits.txt', sep='\t')

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 15 x 2


In [10]:
original_data.head()

,gene,class
0,CTL1,suppressor
1,MET22,suppressor
2,HUR1,suppressor
3,XRN1,suppressor
4,UBP3,suppressor


In [11]:
original_data['gene'] = original_data['gene'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [gene, class, orf]
Index: []


In [19]:
data_switch = {'suppressor': 2, 'accelerator': -2, 'weak accelerator': -1, 'modifier': 0}

In [20]:
original_data['data'] = original_data['class'].apply(lambda x: data_switch[x])

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(15, 1)

# Prepare the final dataset

In [25]:
data = original_data.copy()

In [26]:
dataset_ids = [84]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,84
data_type,value
orf,
YDR176W,-1
YDR323C,-2
YDR392W,0
YER151C,2
YGL168W,2


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,84
,data_type,value
gene_id,orf,
2302,YDR176W,-1
2450,YDR323C,-2
2519,YDR392W,0
879,YER151C,2
2849,YGL168W,2


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id         84           
data_type       value     valuez
gene_id orf                     
2302    YDR176W    -1 -10.611336
2450    YDR323C    -2 -21.441462
2519    YDR392W     0   0.218790
879     YER151C     2  21.879043
2849    YGL168W     2  21.879043

# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16027361...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00, 10.91it/s]

Updating the data_modified_on field...
